### Import Dependencies

In [1]:
! pip -q install bleu
! pip -q install rouge

from rouge import Rouge
from bleu import list_bleu
import pandas as pd
from torchtext.vocab import Vocab
from torch.utils.data import DataLoader
from torchtext.vocab import GloVe
from utils import *
import random
from sklearn.metrics.pairwise import cosine_similarity

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Import and Process Data

In [2]:
# Read Data
df = pd.read_csv('Reviews.csv')
data = extractData(df['Text'], df['Summary'])

# Build Vocabulary
counter = buildCounter(data)    
vocab =  Vocab(counter, specials=['<unk>', '<pad>', '<bos>', '<eos>'], min_freq=500)

# Tokenize Data
tokenized_data = tokenize(data, vocab)

# Train-Test Split
random.shuffle(tokenized_data)
train_data = tokenized_data[:550000]
test_data = tokenized_data[550000:]

In [3]:
# Truncate and pad token sequences, source to length 500, target to 15
def trunc_and_pad(batch):
    src, tgt = [], []
    for s, t in batch:                            
            s_tr = [vocab['<bos>']] + s[:498] + [vocab['<eos>']]
            t_tr = [vocab['<bos>']] + t[:13] + [vocab['<eos>']]
            s_pad = s_tr + [vocab['<pad>']]*(500 - len(s_tr))
            t_pad = t_tr + [vocab['<pad>']]*(15 - len(t_tr))
            src.append(s_pad)
            tgt.append(t_pad)
    return torch.tensor(src).to(device), torch.tensor(tgt).to(device)

# Train and Test DataLoaders
train_loader = DataLoader(train_data, batch_size=16, shuffle=True, collate_fn=trunc_and_pad)
test_loader = DataLoader(test_data, batch_size=1, shuffle=True, collate_fn=trunc_and_pad)

### Training

In [4]:
# Pretrained Glove Embeddings
embedding_size = 100

glove = GloVe(name='6B', dim=embedding_size)

weights = torch.zeros(len(vocab), embedding_size, dtype = torch.float)

for idx, key in enumerate(vocab.stoi.keys()):
    weights[idx] = glove[key]

embedding = nn.Embedding.from_pretrained(weights, freeze=True)

.vector_cache/glove.6B.zip: 862MB [02:40, 5.39MB/s]                               
100%|█████████▉| 399999/400000 [00:19<00:00, 20116.30it/s]


In [8]:
# Training the model
model = train(vocab, embedding, train_loader, test_loader)

<All keys matched successfully>

### Results and Analysis

In [13]:
# Inputs, Targets and Predicted Outputs for some samples
samples = validation_examples(model, test_loader, vocab)

for num, sample in enumerate(samples):
    input, target, result = sample
    print(f"Example {num}:")
    print(f"Input : {reconstruction(input, vocab)}")
    print(f"Target : {reconstruction(target, vocab)}")
    print(f"Result : {reconstruction(result, vocab)}")

In [14]:
# Bleu Score
samples = validation_examples(model, test_loader, vocab,noOfExamples=len(test_loader.dataset))

targets, results = get_reconstructed(samples, vocab)
    
list_bleu(targets, results)

In [ ]:
# Rouge Scores
Rouge().get_scores(results, targets, avg=True)

In [22]:
# Our Metric on Cosine Similarity

similarity = cosine_similarity(weights, weights)
targets, results = get_reconstructed(samples, vocab, return_tokenized=True)

total = 0
nans = 0
for result, target in zip(results, targets):
    sim = bleu_similarity(result, target, similarity) 
    if(sim >= 0):                        # To check not nan
        total += sim

    # This means target is empty, even ROUGE doesn't allow this
    else:             
        nans += 1                        # Exclude this example

total / (len(results) - nans)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:55: RuntimeWarning: Mean of empty slice.


0.4365634628595264